## KD Tree

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode

In [69]:
data = pd.read_csv('/Users/hanifemamgholizadeh/Desktop/patter_recognition/data/tree_classification_dataset.csv')

In [70]:
data.head()

,feature_1,feature_2,feature_3,feature_4,target
0,-3.402839,0.179845,1.432424,0.774566,1
1,-1.902995,-1.241501,1.956614,0.512447,1
2,-1.023094,1.270126,0.782203,-0.785725,0
3,-0.331077,-0.069590,-0.258279,-0.339054,0
4,-3.377452,0.816699,-3.009166,-1.553258,2


In [71]:
y = data['target']
X = data.drop(columns=['target'])
len(y.unique())

3

In [72]:
class TreeNode:
    def __init__(self, left_child=None, right_child=None, parent=None, value=None, feature=None, leaf_list=None):
        self.left_child = left_child
        self.right_child = right_child
        self.parent = parent
        self.value = value
        self.feature = feature
        self.leaf_list = leaf_list if leaf_list is not None else []
        self.is_right_child = False
        self.is_left_child = False

    def is_leaf(self):
        return self.left_child is None and self.right_child is None

In [73]:
def calculate_distance(x1, x2):
    return np.linalg.norm(np.array(x1) - np.array(x2))

In [74]:
def calculate_distance_to_wall(x, node):
    feature = node.feature
    wall_value = node.value
    return abs(x[feature] - wall_value)

In [75]:
def create_KD_tree(X, y, parent_node=None, direction=None):
    if len(X) <= 4 or len(np.unique(y)) == 1:
        # Create a leaf node
        return TreeNode(parent=parent_node, leaf_list=list(zip(X.values, y.values)))

    # Choose a random feature and split on its median
    feature = np.random.choice(X.columns)
    median_value = X[feature].median()

    # Split the data
    left_indices = X[X[feature] <= median_value].index
    right_indices = X[X[feature] > median_value].index

    # Create current node
    node = TreeNode(parent=parent_node, value=median_value, feature=feature)

    if direction == 'left':
        node.is_left_child = True
    elif direction == 'right':
        node.is_right_child = True

    # Recursive construction
    node.left_child = create_KD_tree(X.loc[left_indices], y.loc[left_indices], parent_node=node, direction='left')
    node.right_child = create_KD_tree(X.loc[right_indices], y.loc[right_indices], parent_node=node, direction='right')

    return node


In [76]:
def depth_first_search(tree_node: TreeNode, x, knn_distance, knn=[]):
    if tree_node is None:
        return None
    
    if tree_node.is_leaf():
        leaf_list = tree_node.get_leaf_list()
        for item in leaf_list:
            calculate_distances = [calculate_distance(x, item) for item in leaf_list]
        sorted_indices = np.argsort(calculate_distances)
        knn.append(sorted_indices)
        return tree_node.get_leaf_list()
    
    feature = tree_node.get_feature()
    value = tree_node.get_value()
    
    if calculate_distance_to_wall(x[feature], value) < knn_distance:
        return depth_first_search(tree_node, x, knn_distance, knn)
    else:
        return depth_first_search(tree_node.parent, x)

In [77]:
def depth_first_search(node, x, knn_distance, knn_list):
    if node is None:
        return

    if node.is_leaf():
        for point, label in node.leaf_list:
            dist = calculate_distance(x, point)
            knn_list.append((dist, label))
        return

    feature = node.feature
    value = node.value

    # Choose which subtree to go first
    if x[feature] <= value:
        depth_first_search(node.left_child, x, knn_distance, knn_list)
        if calculate_distance_to_wall(x, node) < knn_distance:
            depth_first_search(node.right_child, x, knn_distance, knn_list)
    else:
        depth_first_search(node.right_child, x, knn_distance, knn_list)
        if calculate_distance_to_wall(x, node) < knn_distance:
            depth_first_search(node.left_child, x, knn_distance, knn_list)

# Query the KD Tre

In [ ]:
def query_KD_tree(tree, x, k=1):
    knn_list = []

    # Initial DFS to fill neighbors
    depth_first_search(tree, x, knn_distance=np.inf, knn_list=knn_list)

    # Sort and return top-k neighbors
    knn_list.sort(key=lambda tup: tup[0])
    return knn_list[:k]

# Example usage
kd_tree = create_KD_tree(X, y)

# Test on one point
test_point = X.iloc[10]
neighbors = query_KD_tree(kd_tree, test_point, k=3)
print("Nearest Neighbors:", neighbors)
print("Test Point prediction:", mode(np.array([preds[1] for preds in neighbors])).mode[0])
print("True Label:", y.iloc[0])

Nearest Neighbors: [(0.0, 1), (0.7395099856826924, 1), (0.7622639206298912, 1)]
Test Point prediction: 1
True Label: 1


/var/folders/n4/8zh_hhf525v6v_00z9234_0h0000gn/T/ipykernel_11386/1229450982.py:18: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  print("Test Point prediction:", mode(np.array([preds[1] for preds in neighbors])).mode[0])


In [80]:
from sklearn.metrics import accuracy_score
def evaluate_model(tree, X, y, k=3):
    predictions = []
    for i in range(len(X)):
        neighbors = query_KD_tree(tree, X.iloc[i], k)
        predicted_label = mode(np.array([preds[1] for preds in neighbors])).mode[0]
        predictions.append(predicted_label)
    
    accuracy = accuracy_score(y, predictions)
    print("Accuracy:", accuracy)
    return accuracy

In [81]:
evaluate_model(kd_tree, X, y, k=3)

/var/folders/n4/8zh_hhf525v6v_00z9234_0h0000gn/T/ipykernel_11386/3337161936.py:6: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  predicted_label = mode(np.array([preds[1] for preds in neighbors])).mode[0]


Accuracy: 0.895


0.895